# The 2017 Eagle Creek Fire near Hood River, Oregon covered more than 48,000 acres and devastated more than half of the Oregon side of the Columbia River Gorge.

![3D extent of Eagle Creek Fire](https://wildfiretoday.com/wp-content/uploads/2017/09/EagleCr_3D2_730pmPDT_9-5-2017.jpg)

Image Source: [Wildfire Today](https://wildfiretoday.com/wp-content/uploads/2017/09/EagleCr_3D2_730pmPDT_9-5-2017.jpg)

![Mountains on Fire](https://advancelocal-adapter-image-uploads.s3.amazonaws.com/image.oregonlive.com/home/olive-media/width2048/img/wildfires/photo/eagle-creek-fire-f163f5cfde990cb7.jpg)

Image Source: [Oregon Live](https://advancelocal-adapter-image-uploads.s3.amazonaws.com/image.oregonlive.com/home/olive-media/width2048/img/wildfires/photo/eagle-creek-fire-f163f5cfde990cb7.jpg)

On a hot and windy fall afternoon on September 2, 2017, two young teens wrecklessly playing with fireworks ignited one of the largest wildfires in northern Oregon's history, descimating near half of the Columbia River Gorge, a famous outdoor recreation area and critical transportation corridor (trains, trucks, and boats).

As the fire quickly grew, 152 hikers were trapped by the flames and eventually rescued while the nearby town of Cascade Locks was evacuated. 2.5 days after ignition, the fire had grown to 10,000 acres and traveled nearly 13 miles in approximately 16 hours, fueled by strong winds. By this time, Interstate 84 has been shutdown and the fire jumped an incredible ~1800 ft across the Columbia River and spread into parts of Washington. Through the blaze, nearly 400 homes were evacuated, hundreds of miles of trail and roadway infracture were destroyed, and historic landmarks, such a [Multinomah Falls](https://www.fs.usda.gov/recarea/crgnsa/recarea/?recid=30026), closely threatened.

The fire was finally contained at the end of November, nearly two months after it started, but continued to smoulder for another 9-months. A total of 48,000 acres burned in the Eagle Creek Fire. While residents were eventually able to return home, the Eagle Creek Trail system remained closed for three years and remains a visible scar on the landscape today. Massive efforts were put into clearing dangerous trees, installing protective fensing, and monitoring for landslide and debris flows. The teen responsible for the fire was sentenced to 1,920 hours of community service with the Forest Service. If this service is not compled in 10 years, the teen will have to pay $36.6 million in restitution.

In [1]:
# Import standard libraries
import logging              # Logging of messages
import os                   # Handles files and directories
import pathlib              # Provides a path class for cross platform use
import subprocess           # Run new code by creating new process
import warnings             # Show warning triggered by module import
from glob import glob       # Creates lists of files for batch processing

# Import third party libraries
import earthpy.appeears as etapp    # Plot and manipulate spatial data
import folium               # Generate interactive maps
import geopandas as gpd     # Work with vector files
import hvplot.pandas        # Used for plotting data
import hvplot.xarray        # Used for plotting maps
import numpy as np          # Used for math operations
import pandas as pd         # Used for working with tabular data
import rioxarray as rxr     # Storage and manipulation of rasters
import xarray as xr         # Work with multidimensional arrays

# Set up logging so AppeearsDownloader will log in notebook
logging.basicConfig(level=logging.INFO)

# Ignore FutureWarning coming from hvplot
#warnings.simplefilter(action='ignore', category=FutureWarning)

ModuleNotFoundError: No module named 'earthpy.appeears'

In [ ]:
# Define/create a directory to store and access data

camp_prj_dir = os.path.join(
    pathlib.Path.home(), 'earth-analytics', 'data', 'camp-fire')
# This defines where the directory can be found

os.makedirs(camp_prj_dir, exist_ok=True)
# This is an instruction to create the directory defined above

camp_prj_dir
# This prints out the directory path name

The API Site for the 2017 Historical Fire Perimeters can be found at:

https://data-nifc.opendata.arcgis.com/datasets/historic-perimeters-2017/api

In [2]:
# Access the data from NIFC website
camp_gdf=gpd.read_file(
    "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis"
    "/rest/services/Historic_Geomac_Perimeters_2017"
    "/FeatureServer/0/query?where=incidentname%20%3D%20"
    "'EAGLE%20CREEK'%20AND%20latest%20%3D%20'Y'&outFields="
    "*&outSR=4326&f=json"
)

camp_gdf

NameError: name 'gpd' is not defined